In [1]:
import time

import numpy as np
import pandas as pd

from preprocessing import build_dataset

pd.options.mode.chained_assignment = None 

In [2]:
# Binary BOAFAB
wl = 3
sr = 22050
flims = (100,2000)
verbose = False # Doubt about use this term
recordings_folder = 'data/INCT20955/raw/recordings/'
annotation_path = 'data/INCT20955/raw/annotations/'
path_save = 'data/datasetv1/binary_boafab_1'
prefix='SAMPLE_'
labels_cols = ['BOAFAB_M','BOAFAB_C'] # looks like multiclass but ends like binary
site=['INCT20955']

start = time.time()
df_boafab_binary_dataset = build_dataset(wav_path=recordings_folder, 
                                                  annotation_path=annotation_path, 
                                                  wl=wl, 
                                                  site=site,
                                                  target_sr=sr, 
                                                  flims=flims,
                                                  path_save=path_save, 
                                                  prefix=prefix, 
                                                  labels_cols=labels_cols,
                                                  verbose=verbose)
end = time.time()
print(round((end - start)/60,1),' minutes')

0it [00:00, ?it/s]/anaconda/envs/gibbons_env/lib/python3.6/site-packages/maad/sound/input_output.py:113: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, s = wavfile.read(filename)
451it [01:44,  3.99it/s]/anaconda/envs/gibbons_env/lib/python3.6/site-packages/maad/sound/input_output.py:113: WavFileWarning: Reached EOF prematurely; finished at 5046272 bytes, expected 5293636 bytes from header.
  fs, s = wavfile.read(filename)
3132it [14:08,  3.69it/s]


Datased created in: data/datasetv1/binary_boafab_1
18  minutes


In [3]:
# Binary PHYCUV
wl = 3
sr = 22050
flims = (300,2000)
verbose=True
recordings_folder = 'data/INCT20955/raw/recordings/'
annotation_path = 'data/INCT20955/raw/annotations/'
path_save='data/datasetv1/binary_phycuv_1'
prefix='SAMPLE_'
labels_cols = ['PHYCUV_M']
site=['INCT20955']

start = time.time()
df_phycuv_binary_dataset = build_dataset(wav_path=recordings_folder, 
                                                  annotation_path=annotation_path, 
                                                  wl=wl, 
                                                  site=site,
                                                  target_sr=sr, 
                                                  flims=flims,
                                                  path_save=path_save, 
                                                  prefix=prefix, 
                                                  labels_cols=labels_cols,
                                                  verbose=verbose)
end = time.time()
print(round((end - start)/60,1),' minutes')

0it [00:00, ?it/s]/anaconda/envs/gibbons_env/lib/python3.6/site-packages/maad/sound/input_output.py:113: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, s = wavfile.read(filename)
340it [01:17,  4.37it/s]


Datased created in: data/datasetv1/binary_phycuv_1
3  minutes


In [5]:
# Multiclass
wl = 3
sr = 22050
flims = (1,10000)
verbose = False # Doubt about use this term
recordings_folder = 'data/INCT20955/raw/recordings/'
annotation_path = 'data/INCT20955/raw/annotations/'
path_save='data/datasetv1/multiclass_1'
prefix='SAMPLE_'
max_duration = 60
labels_cols = ['BOAFAB_M','BOAFAB_C','PHYCUV_M']
site=['INCT20955']

start = time.time()
df_boafab_multiclass_1 = build_dataset(wav_path=recordings_folder, 
                                                  annotation_path=annotation_path, 
                                                  wl=wl, 
                                                  site=site,
                                                  target_sr=sr, 
                                                  flims=flims,
                                                  path_save=path_save, 
                                                  prefix=prefix, 
                                                  labels_cols=labels_cols,
                                                  verbose=verbose)
end = time.time()
print(round((end - start)/60,1),' minutes')

0it [00:00, ?it/s]/anaconda/envs/gibbons_env/lib/python3.6/site-packages/maad/sound/input_output.py:113: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, s = wavfile.read(filename)
536it [01:54,  3.92it/s]/anaconda/envs/gibbons_env/lib/python3.6/site-packages/maad/sound/input_output.py:113: WavFileWarning: Reached EOF prematurely; finished at 5046272 bytes, expected 5293636 bytes from header.
  fs, s = wavfile.read(filename)
3472it [15:23,  3.76it/s]


Datased created in: data/datasetv1/multiclass_1
19.44  minutes
